In [269]:
import tensorflow as tf
import json
import numpy as np
import pandas as pd

from google.cloud import bigquery
from google.cloud import storage

from google.cloud import aiplatform

#https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/custom-tabular-bq-managed-dataset.ipynb

In [270]:
print(tf.__version__)

2.11.0


In [271]:
#!gsutil cp gs://smart-ml-pongthorn/New_Incident.csv data

# !gsutil -m cp -r "gs://tf1-incident-pongthorn/model" .

In [291]:
unUsedColtoPredict=['severity','id','severity_id','severity_name','imported_at']
isLocalModel=True

filePath='../../data/New_Incident.csv'

modelFile_endPoint=1

localFile=True

In [292]:
# def download_table(bq_table_uri: str):
#     # Remove bq:// prefix if present
#     bqclient= bigquery.Client()
#     prefix = "bq://"
#     if bq_table_uri.startswith(prefix):
#         bq_table_uri = bq_table_uri[len(prefix) :]

#     table = bigquery.TableReference.from_string(bq_table_uri)
#     rows = bqclient.list_rows(table)
#     return rows.to_dataframe()

# dfNewData=download_table("pongthorn.SMartML.new_incident")
# dfNewData.to_csv(filePath,index=False)       
# dfNewData.info()


# Load and Map Data Test

In [293]:
if  localFile:
    dfNewData=pd.read_csv(filePath)

dfNewData=dfNewData.sample(10)
print(dfNewData.info())
# dfNewData

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 25 to 19
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         10 non-null     int64  
 1   severity_id                10 non-null     int64  
 2   severity_name              10 non-null     object 
 3   sla                        10 non-null     object 
 4   product_type               10 non-null     object 
 5   brand                      10 non-null     object 
 6   service_type               10 non-null     object 
 7   incident_type              10 non-null     object 
 8   open_to_close_hour         10 non-null     float64
 9   response_to_resolved_hour  10 non-null     float64
 10  imported_at                10 non-null     object 
dtypes: float64(2), int64(2), object(7)
memory usage: 960.0+ bytes
None


In [294]:
mapping_file="incident_sevirity_to_class.json"

def download_map_severity_class_as_dict():
 
    storage_client = storage.Client()
    buckdfNewData = storage_client.bucket("smart-ml-pongthorn")
    blob = bucket.blob(mapping_file)
    blob.download_to_filename(mapping_file)
    with open(mapping_file, 'r') as file:
            return json.loads(file.read())

if  localFile==False:    
    map_sevirity_to_class= download_map_severity_class_as_dict()
else:
   with open(mapping_file, 'r') as json_file:
     map_sevirity_to_class= json.load(json_file)
                                  
                                  
print(map_sevirity_to_class)


{'Cosmatic': 0, 'Minor': 1, 'Major': 2, 'Critical': 3}


In [295]:
dfNewData.insert(2, 'severity', dfNewData['severity_name'].map(map_sevirity_to_class),True)
print(dfNewData.info())
dfNewData

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 25 to 19
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         10 non-null     int64  
 1   severity_id                10 non-null     int64  
 2   severity                   10 non-null     int64  
 3   severity_name              10 non-null     object 
 4   sla                        10 non-null     object 
 5   product_type               10 non-null     object 
 6   brand                      10 non-null     object 
 7   service_type               10 non-null     object 
 8   incident_type              10 non-null     object 
 9   open_to_close_hour         10 non-null     float64
 10  response_to_resolved_hour  10 non-null     float64
 11  imported_at                10 non-null     object 
dtypes: float64(2), int64(3), object(7)
memory usage: 1.0+ KB
None


,id,severity_id,severity,severity_name,sla,product_type,brand,service_type,incident_type,open_to_close_hour,response_to_resolved_hour,imported_at
25,2487,2,2,Major,24x7 6Hrs Resolution Time,Software,SAPB1,Incident,Software,0.083333,0.066667,27:09.0
32,2505,3,1,Minor,24x7 6Hrs Resolution Time,Software,eUnite,Incident,Software,4.316667,4.300000,27:09.0
6,2490,2,2,Major,24x7 4Hrs Resolution Time,Storage,HPE,Incident,Hard Disk Drive Failure,26.133333,4.700000,27:09.0
1,2517,2,2,Major,24x7 4Hrs Resolution Time,Server,HPE,Incident,Memory Failure,6.166667,6.000000,27:09.0
15,2516,3,1,Minor,24x7 4Hrs Resolution Time,Firewall,CheckPoint,Request,General Incident,24.483333,24.016667,27:09.0
34,2508,3,1,Minor,24x7 6Hrs Resolution Time,Software,eUnite,Incident,Software,1.750000,1.733333,27:09.0
22,2506,2,2,Major,24x7 6Hrs Resolution Time,Software,SAPB1,Incident,Software,80.033333,80.016667,27:09.0
26,2480,2,2,Major,24x7 6Hrs Resolution Time,Software,SAPB1,Incident,Software,0.150000,0.133333,27:09.0
17,2434,2,2,Major,24x7 4Hrs Resolution Time,Hardware,Oracle,Incident,Hard Disk Drive Failure,125.000000,119.250000,27:09.0
19,2501,2,2,Major,24x7 4Hrs Resolution Time,Software,VMWare,Incident,Software,5.333333,4.933333,27:09.0


In [296]:
pdPrediction=pd.DataFrame(columns=['_id','_predict_severity'])
pdPrediction

,_id,_predict_severity


# Load Model from Directory to Predict

In [297]:
if modelFile_endPoint==1 :

    if  isLocalModel: # Window Enviroment
        PATH_FOLDER_ARTIFACTS="model"    
    else: # For running on Vertex AI Notebook t
        PATH_FOLDER_ARTIFACTS="gs://tf1-incident-pongthorn/model"

    model = tf.keras.models.load_model(PATH_FOLDER_ARTIFACTS)    
    print(f"Load from {PATH_FOLDER_ARTIFACTS}")
    # model.summary()

    
    for  row_dict in dfNewData.to_dict(orient="records"):
          incident_id=row_dict['id']
          print(f"{incident_id} - {row_dict['severity']}({row_dict['severity_name']})") 
          for key_removed in unUsedColtoPredict:
           row_dict.pop(key_removed)
          # print(row_dict)  

          input_dict = {name: tf.convert_to_tensor([value]) for name, value in row_dict.items()}


          predictionResult = model.predict(input_dict)
          # print(f"{predictionResult}")   
          # maxResult=np.max(arryResult)
          # maxResult=round( maxResult,4)
          # print(f"{maxResult} of {arryResult}")

          prob = tf.nn.softmax(predictionResult)
          prob_pct=(100 * prob)  
          _class = tf.argmax(predictionResult,-1).numpy()[0]
          
          pdPrediction =pd.concat([pdPrediction,pd.DataFrame.from_dict([{'_id':incident_id, '_predict_severity':_class}])] )
          # print(input_dict)  
            
          print(f"{prob_pct} %   as {_class}")     
          print("======================================================================================")

    dfPredictData=pd.merge(dfNewData,pdPrediction,how='inner',left_on='id',right_on='_id')
    dfPredictData=dfPredictData.drop(columns=['_id'])
    dfPredictData=dfPredictData[['id','_predict_severity','severity','severity_name']]


Load from model
2487 - 2(Major)
1/1 [==============================] - 0s 265ms/step
[[19.0795   27.501873 34.28234  19.136284]] %   as 2
2505 - 1(Minor)
1/1 [==============================] - 0s 35ms/step
[[19.080553 27.493492 34.292618 19.133335]] %   as 2
2490 - 2(Major)
1/1 [==============================] - 0s 34ms/step
[[19.614077 27.600952 31.383127 21.40185 ]] %   as 2
2517 - 2(Major)
1/1 [==============================] - 0s 42ms/step
[[18.723211 20.157082 38.59516  22.524544]] %   as 2
2516 - 1(Minor)
1/1 [==============================] - 0s 31ms/step
[[20.952015 20.691626 39.7976   18.558765]] %   as 2
2508 - 1(Minor)
1/1 [==============================] - 0s 34ms/step
[[19.079914 27.49857  34.286396 19.13512 ]] %   as 2
2506 - 2(Major)
1/1 [==============================] - 0s 32ms/step
[[19.101225 27.332178 34.48244  19.084154]] %   as 2
2480 - 2(Major)
1/1 [==============================] - 0s 35ms/step
[[19.079517 27.501738 34.2825   19.13624 ]] %   as 2
2434 - 2(Major)

# Load Model from Online EndPpint to Predict

In [299]:
instances = []
dfX=dfNewData.drop(columns=unUsedColtoPredict)
for index,row in dfX.iterrows():
    dict_item=row.to_dict()
    instance = {}
    for key, value in dict_item.items():
        if key in unUsedColtoPredict:
            continue
        if value is None:
            value = ""
        instance[key] = [value]
        # instance[key] = value
    instances.append(instance)

print(len(instances))
print(instances[0])

10
{'sla': ['24x7 6Hrs Resolution Time'], 'product_type': ['Software'], 'brand': ['SAPB1'], 'service_type': ['Incident'], 'incident_type': ['Software'], 'open_to_close_hour': [0.083333333], 'response_to_resolved_hour': [0.066666667]}


In [300]:
isEndpointAvaiable=False


try:
    PROJECT_NUMBER="pongthorn"
    ENDPOINT_ID="2704182877817405440"
    REGION="asia-southeast1"
    endpoint = aiplatform.Endpoint(
    endpoint_name=f"projects/{PROJECT_NUMBER}/locations/{REGION}/endpoints/{ENDPOINT_ID}")
    print(endpoint)
    isEndpointAvaiable=True



except Exception as error:
    print(error.message)

Endpoint projects/pongthorn/locations/asia-southeast1/endpoints/2704182877817405440 is not found.


In [301]:
if modelFile_endPoint==2 and isEndpointAvaiable:


    for data in dfNewData.to_dict(orient="records"):
        incident_id=data['id']
        severity=data['severity']
        severity_name=data['severity_name']
        #{(a if condition else b): value for key, value in dict.items()}
        request_data = {key : [value] for key,value in data.items() }
        for key_removed in unUsedColtoPredict:
           request_data.pop(key_removed)
        print(request_data)

        response = endpoint.predict([request_data])
        y_predicted = np.argmax(response.predictions, axis=1)
        predictionResult=response[0][0]
        print(predictionResult)
        print(y_predicted)

        pdPrediction =pd.concat([pdPrediction,pd.DataFrame.from_dict([{'_id':incident_id, '_predict_severity':y_predicted[0]}])] )
        print("======================================================================================")


    dfPredictData=pd.merge(dfNewData,pdPrediction,how='inner',left_on='id',right_on='_id')
    dfPredictData=dfPredictData.drop(columns=['_id'])
    dfPredictData=dfPredictData[['id','_predict_severity','severity','severity_name']]


# Classification Rerport

In [302]:
from sklearn.metrics import confusion_matrix,classification_report


In [303]:
dfPredictData

,id,_predict_severity,severity,severity_name
0,2487,2,2,Major
1,2505,2,1,Minor
2,2490,2,2,Major
3,2517,2,2,Major
4,2516,2,1,Minor
5,2508,2,1,Minor
6,2506,2,2,Major
7,2480,2,2,Major
8,2434,2,2,Major
9,2501,2,2,Major


In [304]:
className=list(set().union(list(dfPredictData['severity'].unique()),list(dfPredictData['_predict_severity'].unique())))
actualClass=[  f'actual-{x}' for x in  className]
predictedlClass=[  f'predcited-{x}' for x in className]

y_true=list(dfPredictData['severity'])
y_pred=list(dfPredictData['_predict_severity'])
cnf_matrix = confusion_matrix(y_true,y_pred)
cnf_matrix

# #index=actual , column=prediction
cm_df = pd.DataFrame(cnf_matrix,
                     index = actualClass, 
                     columns = predictedlClass)
cm_df

,predcited-1,predcited-2
actual-1,0,3
actual-2,0,7


In [305]:

print(classification_report(y_true, y_pred, labels=className))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         3
           2       0.70      1.00      0.82         7

    accuracy                           0.70        10
   macro avg       0.35      0.50      0.41        10
weighted avg       0.49      0.70      0.58        10



D:\ProgramData\Anaconda3\envs\dev_google\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ProgramData\Anaconda3\envs\dev_google\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ProgramData\Anaconda3\envs\dev_google\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
